# Option A - Generate Sphere Manually

In [ ]:
out_dir = '/Volumes/OneTouch/datasets/VOSS_STUDIES_TMS/HEBSCHER_TMS/derivatives'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [ ]:
#Enter coordinates of sphere
x = -30
y = -51
z = -9
radius=5

Generate the Sphere at The Coordinate

In [ ]:
import nibabel as nib
import numpy as np
brain_img = nib.load(mask_path)
mni_arr = np.array([x,y,z])

In [ ]:
from calvin_utils.nifti_utils.generate_spherical_roi import calculate_voxelwise_mni_coords, calculate_pairwise_distance, generate_sphere_mask, points_within_sphere, save_mask, mask_within_brain
mni_coords_brain = calculate_voxelwise_mni_coords(brain_img)
distances = calculate_pairwise_distance(mni_arr, mni_coords_brain)
points_in_sphere = points_within_sphere(distances, radius)
sphere_mask = generate_sphere_mask(points_in_sphere, brain_img)
sphere_mask = mask_within_brain(sphere_mask, brain_img)
img = save_mask(sphere_mask, brain_img, mni_arr, out_dir)
img

In [ ]:
import nilearn
nilearn.datasets.fetch_atlas_talairach('ba', data_dir='/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/talairach_spaces', verbose=1)

# Option B - Generate Spheres from a CSV

In [1]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/Jung_TMS_AD/derivatives'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [2]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/Jung_TMS_AD/metadata/master_list.csv'
sheet = None #'master_list_proper_subjects'

In [3]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,tms_id,mni_x,mni_y,mni_z,visit,control0_test1,c0_t1,c0_t1_1,mmse_,moca_,...,swmte4,swmte468,swmte6,swmte8,swmwe12,swmwe4,swmwe468,swmwe6,swmwe8,rownumber
0,S05,-52.0,-52.0,37.0,0,0.0,control,0,25.0,23.0,...,0.0,25.0,7.0,18.0,0.0,0.0,0.0,0.0,0.0,14.0
1,S05,NaN,NaN,NaN,1,NaN,control,0,22.0,21.0,...,3.0,24.0,7.0,14.0,1.0,0.0,1.0,1.0,0.0,15.0
2,S05,NaN,NaN,NaN,2,NaN,control,0,23.0,22.0,...,4.0,25.0,7.0,14.0,1.0,0.0,2.0,1.0,1.0,16.0
3,S06,-58.0,-66.0,26.0,0,0.0,control,0,19.0,17.0,...,1.0,26.0,5.0,20.0,2.0,0.0,4.0,1.0,3.0,4.0
4,S06,NaN,NaN,NaN,1,NaN,control,0,21.0,17.0,...,2.0,20.0,9.0,9.0,3.0,0.0,2.0,2.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,S48,NaN,NaN,NaN,1,NaN,treatment,1,27.0,23.0,...,2.0,29.0,9.0,18.0,1.0,0.0,3.0,0.0,3.0,85.0
104,S48,NaN,NaN,NaN,2,NaN,treatment,1,27.0,25.0,...,3.0,27.0,8.0,16.0,1.0,0.0,2.0,0.0,2.0,86.0
105,S50,-47.0,-59.0,39.0,0,1.0,treatment,1,23.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,S50,NaN,NaN,NaN,1,NaN,treatment,1,24.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



# Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [4]:
data_df.columns

Index(['tms_id', 'mni_x', 'mni_y', 'mni_z', 'visit', 'control0_test1', 'c0_t1',
       'c0_t1_1', 'mmse_', 'moca_',
       ...
       'swmte4', 'swmte468', 'swmte6', 'swmte8', 'swmwe12', 'swmwe4',
       'swmwe468', 'swmwe6', 'swmwe8', 'rownumber'],
      dtype='object', length=191)

In [5]:
drop_list = ['tms_id', 'mni_x']

In [6]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)

Create x,y,z columns if not present

In [42]:
coord_col = 'stimsite_TLRC'

Edit column coord list

In [43]:
# import numpy as np 
# import pandas as pd
# try:
#     data_df[coord_col] = data_df[coord_col].str.replace(',', '', regex=False)
#     data_df[coord_col] = data_df[coord_col].apply(lambda x: np.array(x.split(), dtype=float))
# except:
#     print('column already prepared')
# data_df

In [44]:
# # Split the column into x, y, z values while skipping invalid rows
# def extract_coords(row):
#     try:
#         # Attempt to split and convert to floats
#         coords = [float(value) for value in str(row).replace(',', ' ').split()]
#         if len(coords) == 3:  # Ensure exactly 3 values
#             return coords
#     except ValueError:
#         pass
#     # Return None for invalid rows
#     return [None, None, None]

# # Apply the function and create the new columns
# data_df[['x_mni_cathode', 'y_mni_cathode', 'z_mni_cathode']] = pd.DataFrame(
#     data_df[coord_col].apply(extract_coords).tolist(),
#     index=data_df.index
# )

# data_df

Optional - Convert Coordinates to Different Space

In [ ]:
# Perform the conversion
from calvin_utils.nifti_utils.convert_coordinates import convert_coordinates_in_df
df_converted = convert_coordinates_in_df(data_df, 'x_tal', 'y_tal', 'z_tal', 'talairach_to_mni')
display(df_converted)

Save changes to CSV if needed

In [46]:
# data_df.to_csv(input_csv_path.split('.csv')[0] + '_updated_coords.csv')

Generate spheres given x, y, and z columns

In [7]:
data_df

,tms_id,mni_x,mni_y,mni_z,visit,control0_test1,c0_t1,c0_t1_1,mmse_,moca_,...,swmte4,swmte468,swmte6,swmte8,swmwe12,swmwe4,swmwe468,swmwe6,swmwe8,rownumber
0,S05,-52.0,-52.0,37.0,0,0.0,control,0,25.0,23.0,...,0.0,25.0,7.0,18.0,0.0,0.0,0.0,0.0,0.0,14.0
3,S06,-58.0,-66.0,26.0,0,0.0,control,0,19.0,17.0,...,1.0,26.0,5.0,20.0,2.0,0.0,4.0,1.0,3.0,4.0
6,S09,-54.0,-65.0,11.0,0,1.0,treatment,1,27.0,25.0,...,2.0,24.0,8.0,14.0,0.0,0.0,0.0,0.0,0.0,7.0
9,S10,-47.0,-40.0,63.0,0,0.0,control,0,25.0,24.0,...,3.0,28.0,11.0,14.0,0.0,0.0,3.0,3.0,0.0,1.0
12,S11,-62.0,-49.0,35.0,0,1.0,treatment,1,22.0,16.0,...,2.0,21.0,6.0,13.0,10.0,0.0,0.0,0.0,0.0,10.0
15,S12,-42.0,-60.0,29.0,0,1.0,treatment,1,23.0,23.0,...,4.0,54.0,19.0,31.0,16.0,1.0,24.0,10.0,13.0,20.0
18,S14,-33.0,-77.0,34.0,0,0.0,control,0,26.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,S15,-49.0,-76.0,27.0,0,1.0,treatment,1,29.0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,S17,-53.0,-58.0,26.0,0,0.0,control,0,20.0,16.0,...,2.0,20.0,6.0,12.0,21.0,0.0,1.0,1.0,0.0,26.0
27,S18,-38.0,-57.0,63.0,0,0.0,control,0,20.0,19.0,...,3.0,43.0,12.0,28.0,17.0,0.0,11.0,2.0,9.0,29.0


In [8]:
radius = 4
xcol = 'mni_x'
ycol = 'mni_y'
zcol = 'mni_z'
subject_col = 'tms_id'
session_col = None

In [9]:
from calvin_utils.nifti_utils.generate_spherical_roi import generate_spherical_rois_from_df
generate_spherical_rois_from_df(data_df, xcol, ycol, zcol, subject_col, radius, mask_path, out_dir, session_col=session_col, project_on_to_brain=True)

/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/nilearn/image/resampling.py:805: UserWarning: Casting data from int32 to float32
  return resample_img(
/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/nilearn/image/resampling.py:297: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(
/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/nilearn/image/resampling.py:805: UserWarning: Casting data from int32 to float32
  return resample_img(
/Users/cu135/.virtualenvs/Python_3.7.7_nimlab/lib/python3.11/site-packages/nilearn/image/resampling.py:297: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn(
/Users/cu135/.virtualenvs/Python_3.7.7_nimla

Done
- Calvin

In [1]:

def tal2mni_matrix():
    """
    Defines the inverted transformation matrices for Talairach to MNI conversion.
    Returns:
        T2M (dict): Dictionary containing the inverse rotation and zoom matrices.
    """
    T2M = {}

    # Inverted rotation matrix
    T2M['rotn_inv'] = np.linalg.inv(np.array([
        [1.0000,  0.0000,   0.0000,   0.0000],
        [0.0000,  0.9988,   0.0500,   0.0000],
        [0.0000, -0.0500,   0.9988,   0.0000],
        [0.0000,  0.0000,   0.0000,   1.0000]
    ]))

    # Inverted zoom matrix for above AC/PC line
    T2M['upZ_inv'] = np.linalg.inv(np.array([
        [0.9900,  0.0000,   0.0000,   0.0000],
        [0.0000,  0.9700,   0.0000,   0.0000],
        [0.0000,  0.0000,   0.9200,   0.0000],
        [0.0000,  0.0000,   0.0000,   1.0000]
    ]))

    # Inverted zoom matrix for below AC/PC line
    T2M['downZ_inv'] = np.linalg.inv(np.array([
        [0.9900,  0.0000,   0.0000,   0.0000],
        [0.0000,  0.9700,   0.0000,   0.0000],
        [0.0000,  0.0000,   0.8400,   0.0000],
        [0.0000,  0.0000,   0.0000,   1.0000]
    ]))

    return T2M

def tal2mni(inpoints):
    """
    Converts Talairach coordinates to MNI coordinates.
    
    Args:
        inpoints (numpy.ndarray): Nx3 or 3xN array of Talairach coordinates (X, Y, Z).
    
    Returns:
        numpy.ndarray: Converted MNI coordinates.
    """
    if inpoints.shape[0] != 3:
        raise ValueError("Input must be a Nx3 or 3xN matrix")
    T2M = tal2mni_matrix()
    
    inpoints = np.hstack((inpoints, 1))
    inpoints = np.reshape(inpoints, (4,1))
    
    # Identify points above and below the AC/PC line
    outpoints = np.zeros_like(inpoints)
    if inpoints[2, :] < 0:
        outpoints[:] = (T2M['downZ_inv'] @ T2M['rotn_inv'] @ inpoints[:])
    else:
        outpoints[:] = (T2M['upZ_inv'] @ T2M['rotn_inv'] @ inpoints[:])
        
    # Return only the X, Y, Z coordinates
    return outpoints[:3].flatten()

In [5]:
import numpy as np
tal2mni(np.array([-50, 30, 36]))

array([-50,  29,  40])